In [21]:
import os
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import OneClassSVM
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input
import joblib

# Path to your image dataset
image_path = r"C:\\Users\\LENOVO\\Desktop\\Data_set"


# Parameters
image_size = (224, 224)  # Image size for EfficientNet
batch_size = 32          # Batch size for processing

In [22]:
# Step 1: Load and preprocess images
def load_images(image_path, image_size):
    image_files = [os.path.join(image_path, f) for f in os.listdir(image_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    images = []
    for file in image_files:
        img = load_img(file, target_size=image_size)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)  # Preprocess for EfficientNet
        images.append(img_array)
    return np.array(images)

print("Loading images...")
images = load_images(image_path, image_size)
print(f"Loaded {len(images)} images.")

Loading images...
Loaded 2928 images.


In [23]:
# Step 2: Extract features using EfficientNet
print("Extracting features using EfficientNet...")

base_model =  EfficientNetB7(weights='imagenet', include_top=False, pooling='avg')
features = base_model.predict(images, batch_size=batch_size)
print(f"Extracted features shape: {features.shape}")

Extracting features using EfficientNet...
92/92 ━━━━━━━━━━━━━━━━━━━━ 254s 3s/step
Extracted features shape: (2928, 2560)


In [24]:
# Step 3: Hyperparameter tuning for One-Class SVM
print("Tuning One-Class SVM hyperparameters...")

from sklearn.model_selection import RandomizedSearchCV


from sklearn.model_selection import RandomizedSearchCV
# Custom scoring function for GridSearchCV
def custom_scorer(estimator, X, y=None):
    predictions = estimator.predict(X)
    # Assume all data is normal (label = 1) for training
    ground_truth = np.ones(len(predictions))
    # Compute accuracy of normal predictions
    return np.sum(predictions == 1) / len(predictions)

# Define hyperparameter grid
param_dist = {
    'nu': [0.01, 0.05, 0.1],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1]
}

# Use GridSearchCV to find the best parameters
scorer = custom_scorer  # Use the custom scorer
oc_svm = RandomizedSearchCV(OneClassSVM(kernel='rbf'), param_dist, n_iter=15, scoring=scorer, cv=3, random_state=42)
oc_svm.fit(features)

Tuning One-Class SVM hyperparameters...


RandomizedSearchCV(cv=3, estimator=OneClassSVM(), n_iter=15,
                   param_distributions={'gamma': ['scale', 'auto', 0.001, 0.01,
                                                  0.1],
                                        'nu': [0.01, 0.05, 0.1]},
                   random_state=42,
                   scoring=<function custom_scorer at 0x000001359AFFC040>)

In [25]:
# Step 4: Save the trained model for later use
model_path = "one_class_svm_model.pkl"
joblib.dump(oc_svm, model_path)
print(f"Model saved to {model_path}")

Model saved to one_class_svm_model.pkl


In [28]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input
import joblib

# Paths
trained_model_path = "one_class_svm_model.pkl"
test_image_path = r"C:\\Users\\LENOVO\\Desktop\\DSGP_\\New folder\\Untitled47.jpg"  # Replace with your test image or folder

# Parameters
image_size = (224, 224)

# Load the trained model
print("Loading trained model...")
oc_svm = joblib.load(trained_model_path)

# Load the pre-trained EfficientNet model
print("Loading EfficientNet model for feature extraction...")
base_model = EfficientNetB7(weights='imagenet', include_top=False, pooling='avg')

# Prediction function
def predict(image_path, oc_svm, base_model):
    # Preprocess the image
    img = load_img(image_path, target_size=image_size)
    img_array = preprocess_input(img_to_array(img))
    img_array = np.expand_dims(img_array, axis=0)
    
    # Extract features
    feature = base_model.predict(img_array)
    
    # Predict using One-Class SVM
    prediction = oc_svm.predict(feature)
    return "Normal" if prediction[0] == 1 else "Anomaly"

# Check if test_image_path is a file or directory
if os.path.isfile(test_image_path):
    print(f"Testing single image: {test_image_path}")
    result = predict(test_image_path, oc_svm, base_model)
    print(f"Image: {test_image_path} -> Prediction: {result}")
elif os.path.isdir(test_image_path):
    print("Testing images in directory...")
    test_images = [os.path.join(test_image_path, f) for f in os.listdir(test_image_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for image_file in test_images:
        result = predict(image_file, oc_svm, base_model)
        print(f"Image: {image_file} -> Prediction: {result}")
else:
    print("Invalid path! Please provide a valid image file or directory.")


Loading trained model...
Loading EfficientNet model for feature extraction...
Testing single image: C:\\Users\\LENOVO\\Desktop\\DSGP_\\New folder\\Untitled47.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Image: C:\\Users\\LENOVO\\Desktop\\DSGP_\\New folder\\Untitled47.jpg -> Prediction: Anomaly
